# 설정

In [1]:
# print('### OS ###')
# !cat /etc/issue.net

# print('### CPU ###')
# !cat /proc/cpuinfo

# print('### MEM ###')
# !cat /proc/meminfo

# print('### DISK ###')
# !df -h

# print('### GPU ###')
# !nvidia-smi

In [2]:
import tensorflow as tf
print(tf.__version__)

import torch
import torchvision
import torchtext
print(torch.__version__)

2.4.1
1.7.1+cu101


In [1]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender

  Created wheel for colabgymrender: filename=colabgymrender-1.0.8-cp37-none-any.whl size=2511 sha256=a56afe9160862b4825e740dbc08989ec22d3b77b907cc6601cfde3a8981c6b13
  Stored in directory: /root/.cache/pip/wheels/8a/d1/e8/ef1d4f6e536cc6b965b28c859ccd2a7f7ab123b94ef4007712
Successfully built colabgymrender


# 랜덤 시도

In [4]:
# 초기 설정 (colab 렌더링 포함)
import gym
from colabgymrender.recorder import Recorder

env = gym.make('CartPole-v0')

directory = './video'
env = Recorder(env, directory)


In [ ]:
# 랜덤 시도
random_episodes = 0
reward_sum = 0
env.reset()

while random_episodes < 10:
  action = env.action_space.sample()
  observation, reward, done, _ = env.step(action)
  print(observation, reward, done)
  reward_sum += reward
  if done:
    random_episodes += 1
    print("reward for this episodes was:", reward_sum)
    reward_sum = 0
    env.play()
    env.reset()

# Q 네트워크

In [2]:
# 초기 설정 (텐서플로우1 사용, colab 렌더링 포함)
%tensorflow_version 1.x
import tensorflow as tf

import numpy as np
import gym
env = gym.make('CartPole-v0') 

from colabgymrender.recorder import Recorder

directory = './video'
env = Recorder(env, directory)

######
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
learning_rate = 1e-1

X = tf.placeholder(shape=[None, input_size],
                   dtype=tf.float32, name="input_x")  # 입력 정의

# weight 정의
W1 = tf.get_variable("W1", shape=[
                     input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())

Qpred = tf.matmul(X, W1)  # 네트워크 출력 (예측) 정의
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)  # 레이블 정의

loss = tf.reduce_sum(tf.square(Y - Qpred))  # 손실함수 정의
train = tf.train.AdamOptimizer(
    learning_rate=learning_rate).minimize(loss)  # 훈련 모델 정의

dis = .99
num_episodes = 100
rList = []

TensorFlow 1.x selected.
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2351104/45929032 bytes (5.1%)5914624/45929032 bytes (12.9%)9789440/45929032 bytes (21.3%)13615104/45929032 bytes (29.6%)17235968/45929032 bytes (37.5%)20406272/45929032 bytes (44.4%)24027136/45929032 bytes (52.3%)27770880/45929032 bytes (60.5%)31080448/45929032 bytes (67.7%)34725888/45929032 bytes (75.6%)38404096/45929032 bytes (83.6%)42147840/45929032 bytes (91.8%)

In [3]:

init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  for i in range(num_episodes):
    s = env.reset()
    e = 1. / ((i/10) + 1)
    rAll = 0
    done = False
    step_count = 0

    while not done:
      step_count += 1
      x = np.reshape(s, [1, input_size])  # 상태 s를 전처리해서 x로 만든다.
      Qs = sess.run(Qpred, feed_dict={X: x})
      if np.random.rand(1) < e:  # e-greedy 방식
        a = env.action_space.sample()
      else:
        a = np.argmax(Qs)

      s1, reward, done, _ = env.step(a)  # 액션 수행
      if done:
        Qs[0, a] = -100  # 넘어지면 그 행동에 -100 적용
      else:
        x1 = np.reshape(s1, [1, input_size])  # s를 전처리 해서 x로
        Qs1 = sess.run(Qpred, feed_dict={X: x1})
        Qs[0, a] = reward + dis * np.max(Qs1)  # 계속 써온 그 갱신 공식

      sess.run(train, feed_dict={X: x, Y: Qs})  # 입력 x와 레이블 Qs를 기준으로 모델을 학습시킨다.
      s = s1  # 다음 상태로 전이

    rList.append(step_count)  # 현재 에피소드에서 버틴 횟수를 기록
    print("Episode: {} steps: {}".format(i, step_count))
    # 최근 10번의 평균이 500을 초과하면 훈련 종료
    if len(rList) > 10 and np.mean(rList[-10:]) > 500:
      break

  # 학습을 마치고 이제 테스트하기 위해 상태를 초기화 한다.
  observation = env.reset()
  reward_sum = 0
  while True: # 실제로 시뮬레이션 한다.
    # env.render()

    x = np.reshape(observation, [1, input_size])
    Qs = sess.run(Qpred, feed_dict={X: x})
    a = np.argmax(Qs)

    observation, reward, done, _ = env.step(a)
    reward_sum += reward
    if done:
      print("Total score: {}".format(reward_sum))
      env.play()
      break


Episode: 0 steps: 16
Episode: 1 steps: 9
Episode: 2 steps: 32
Episode: 3 steps: 19
Episode: 4 steps: 15
Episode: 5 steps: 32
Episode: 6 steps: 18
Episode: 7 steps: 10
Episode: 8 steps: 9
Episode: 9 steps: 10
Episode: 10 steps: 16
Episode: 11 steps: 12
Episode: 12 steps: 9
Episode: 13 steps: 11
Episode: 14 steps: 11
Episode: 15 steps: 53
Episode: 16 steps: 37
Episode: 17 steps: 67
Episode: 18 steps: 125
Episode: 19 steps: 33
Episode: 20 steps: 21
Episode: 21 steps: 41
Episode: 22 steps: 25
Episode: 23 steps: 60
Episode: 24 steps: 47
Episode: 25 steps: 21
Episode: 26 steps: 14
Episode: 27 steps: 36
Episode: 28 steps: 101
Episode: 29 steps: 22
Episode: 30 steps: 46
Episode: 31 steps: 24
Episode: 32 steps: 34
Episode: 33 steps: 24
Episode: 34 steps: 27
Episode: 35 steps: 15
Episode: 36 steps: 27
Episode: 37 steps: 16
Episode: 38 steps: 46
Episode: 39 steps: 29
Episode: 40 steps: 18
Episode: 41 steps: 12
Episode: 42 steps: 12
Episode: 43 steps: 9
Episode: 44 steps: 9
Episode: 45 steps: 9
Ep

 40%|████      | 29/72 [00:00<00:00, 283.26it/s]

Total score: 70.0


 99%|█████████▊| 71/72 [00:00<00:00, 301.96it/s]
